In [16]:
import os
import langchain
import constants
from langchain_core.prompts import PromptTemplate, FewShotPromptWithTemplates

## Creating Prompt

In [ ]:
demo_template = """Act as a financial advisor for the people.
In an easy was, explain the basics of {financial_concept}."""

prompt = PromptTemplate(
    input_variable = ['financial_concept'],
    template = demo_template
    )

prompt.format(financial_concept = 'income tax')

'Act as a financial advisor for the people.\nIn an easy was, explain the basics of income tax.'

## LLM

In [5]:
from langchain_openai import OpenAI

llm = OpenAI(openai_api_key=constants.openai_key, temperature= 0.7)

chain1 = prompt | llm

In [7]:
chain1.invoke("GDP")

"\n\nGDP stands for Gross Domestic Product. It is a measure of the total value of all goods and services produced within a country's borders in a specific period of time, usually a year. It is often used as a measure of a country's economic health and growth.\n\nThere are two main ways to calculate GDP: the expenditure approach and the income approach. The expenditure approach adds up all the money spent by households, businesses, and the government on goods and services. This includes things like consumer purchases, investments, and government spending. The income approach adds up all the income earned by workers, businesses, and the government through wages, profits, and taxes.\n\nGDP is important because it gives us an idea of how much a country is producing and how its economy is doing. A high GDP means that a country is producing a lot and its economy is growing, while a low GDP can indicate a struggling economy.\n\nHowever, GDP does not tell the whole story of a country's economi

## Language Translation

In [10]:
from langchain_core.output_parsers import StrOutputParser

template = """In an easy way translate the sentences '{sentence} into {target_language}'"""
language_prompt = PromptTemplate(
    input_variable = ['sentence', 'target_language'],
    template = template
    )

In [9]:
language_prompt.format(sentence = "How are you", target_language = "Bangla")

"In an easy way translate the sentences 'How are you into Bangla'"

In [13]:
chain2 = language_prompt | llm | StrOutputParser()

In [14]:
response = chain2.invoke({
    "sentence" : "I want to be a Data Scienctist",
    "target_language" : "Bengali"
})

In [15]:
print(response)


আমি একজন ডেটা সাইন্টিস্ট হতে চাই


## Few Shot Prompting

In [18]:
#first, create the list of few shot examples

examples = [
    {"word" : "happy",
     "antonym" : "sad"},
    {"word": "tall",
     "antonym" : "short"}
]
#next, we specify the template to format the examples we have provided.
#we use the `PromptTemplate` class for this 

example_formatted_template = """Word : {word}
Antonym : {antonym}"""

example_prompt = PromptTemplate(
    input_variables= ['word', 'antonym'],
    template = example_formatted_template
)

In [21]:
few_shot_prompt = FewShotPromptWithTemplates(
    #These are the examples we want to insert into the prompt
    examples= examples,
    #This is how we want to format our examples
    example_prompt = example_prompt,
    #The prefix is some text that goes before the examples in the prompt.
    prefix = PromptTemplate.from_template("Give the antonym of every input\n"),
    #The suffix is some text that goes after the examples in the prompt.
    #Usually, this is where the user input would go.
    suffix= PromptTemplate.from_template("Word: {input} \n Antonym : "),
    #The input variables are the variables that overall prompt expects.
    input_variables=["input"],
    #The example_separator is the string we will use to join the prefix, examples
    example_separator= "\n"
)

In [22]:
print(few_shot_prompt.format(input = 'Big'))

Give the antonym of every input

Word : happy
Antonym : sad
Word : tall
Antonym : short
Word: Big 
 Antonym : 


In [23]:
chain3 = few_shot_prompt | llm | StrOutputParser()

In [ ]:
response = chain3.invoke({
    "input" : "Extra"
    
})

In [ ]:
print(response)

Small
